In [2]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/HCIb25-26/data/*.pkl .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz

In [4]:
#!gunzip cc.en.300.vec.gz
#!gunzip cc.uk.300.vec.gz

In [5]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.bin.gz
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

In [1]:
!pip install gensim

In [2]:
from gensim.models import KeyedVectors
import pandas as pd
import pickle
import numpy as np


In [3]:
#lang1_embeddings = KeyedVectors.load_word2vec_format("cc.en.300.vec", binary=False)

# Unpickling from a file
with open('en_vec.pkl', 'rb') as f:
    lang1_embeddings = pickle.load(f)

In [4]:
# 1 - en, 2 - uk
#lang2_embeddings = KeyedVectors.load_word2vec_format("cc.uk.300.vec", binary=False)

# Unpickling from a file
with open('uk_vec.pkl', 'rb') as f:
    lang2_embeddings = pickle.load(f)

In [5]:
"""

# Pickling to a file
with open('en_vec.pkl', 'wb') as f:
    pickle.dump(lang1_embeddings, f)

# Pickling to a file
with open('uk_vec.pkl', 'wb') as f:
    pickle.dump(lang2_embeddings, f)
"""


"\n\n# Pickling to a file\nwith open('en_vec.pkl', 'wb') as f:\n    pickle.dump(lang1_embeddings, f)\n\n# Pickling to a file\nwith open('uk_vec.pkl', 'wb') as f:\n    pickle.dump(lang2_embeddings, f)\n"

In [6]:
#!pip install fasttext

In [7]:
#import fasttext.util


In [8]:
#fasttext.util.download_model('uk', if_exists='ignore')  # Ukrainian
#fasttext.util.download_model('en', if_exists='ignore')  # English

In [9]:
# 1 - en, 2 - uk

#lang1_embeddings = fasttext.load_model('cc.en.300.bin')
#lang2_embeddings = fasttext.load_model('cc.uk.300.bin')

In [10]:
!wget https://dl.fbaipublicfiles.com/arrival/dictionaries/en-uk.0-5000.txt
!wget https://dl.fbaipublicfiles.com/arrival/dictionaries/en-uk.5000-6500.txt

--2025-11-06 07:52:18--  https://dl.fbaipublicfiles.com/arrival/dictionaries/en-uk.0-5000.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 54.240.184.75, 54.240.184.92, 54.240.184.91, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|54.240.184.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232454 (227K) [text/plain]
Saving to: ‘en-uk.0-5000.txt.1’

en-uk.0-5000.txt.1  100%[===================>] 227.01K  --.-KB/s    in 0.03s   

2025-11-06 07:52:18 (6.50 MB/s) - ‘en-uk.0-5000.txt.1’ saved [232454/232454]

--2025-11-06 07:52:18--  https://dl.fbaipublicfiles.com/arrival/dictionaries/en-uk.5000-6500.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 54.240.184.75, 54.240.184.92, 54.240.184.91, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|54.240.184.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51769 (51K) [text/plain]
Saving to: ‘en-uk.5000-6500.txt.1’

en-uk.500

In [11]:
def get_dict(file_name):
    my_file = pd.read_csv(file_name, delimiter='\t')
    etof = {}
    for i in range(len(my_file)):
        lang1 = my_file.loc[i][0]
        lang2 = my_file.loc[i][1]
        etof[lang1] = lang2

    return etof


In [12]:
l1_l2_train = get_dict('en-uk.0-5000.txt')
l1_l2_test = get_dict('en-uk.5000-6500.txt')


/tmp/ipython-input-2844298481.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lang1 = my_file.loc[i][0]
/tmp/ipython-input-2844298481.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lang2 = my_file.loc[i][1]
/tmp/ipython-input-2844298481.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lang1 = my_file.loc[i][0]
/tmp/ipython-input-2844298481.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. 

In [13]:
def get_matrices(l1_l2, l2_embeddings, l1_embeddings):
    """
    Вхід:
        l1_l2: словник перекладів з мови-1 на мову-2
        l2_embeddings: KeyedVectors векторів слів мови-2
        l1_embeddings: KeyedVectors векторів слів мови-1
    Вихід:
        X: матриця, де кожен рядок є вектором слова мови-1
        Y: матриця, де кожен рядок є вектором перекладу цього слова мовою-2
    """
    X_l = list()
    Y_l = list()

    l1_set = set(l1_embeddings.index_to_key)
    l2_set = set(l2_embeddings.index_to_key)

    for l1_word, l2_word in l1_l2.items():
        if l1_word in l1_set and l2_word in l2_set:
            X_l.append(l1_embeddings[l1_word])
            Y_l.append(l2_embeddings[l2_word])

    X = np.asarray(X_l)
    Y = np.asarray(Y_l)
    return X, Y



In [14]:
X_train, Y_train = get_matrices(l1_l2_train, lang2_embeddings, lang1_embeddings)


In [15]:
Y_train.shape

(4666, 300)

In [16]:
def compute_loss(X, Y, R):
    """
    Вхід:
        X: матриця векторів слів мови-1 (розмірність m x n)
        Y: матриця векторів слів мови-2 (розмірність m x n)
        R: матриця перетворення (розмірність n x n)
    Вихід:
        loss: значення функції втрат
    """
    m = X.shape[0]
    diff = np.dot(X, R) - Y
    loss = np.sum(diff**2) / m
    return loss


In [17]:
def compute_gradient(X, Y, R):
    """
    Вхід:
        X: матриця векторів слів мови-1 (розмірність m x n)
        Y: матриця векторів слів мови-2 (розмірність m x n)
        R: матриця перетворення (розмірність n x n)
    Вихід:
        gradient: матриця градієнту функції втрат
    """
    m = X.shape[0]
    gradient = np.dot(X.T, np.dot(X, R) - Y) * (2 / m)
    return gradient


In [18]:
def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003, verbose=True,
                     compute_loss=compute_loss, compute_gradient=compute_gradient):
    """
    Вхід:
        X: матриця (розмірність m x n), кожен рядок - вектор слова мови-1
        Y: матриця (розмірність m x n), кожен рядок - вектор перекладу мовою-2
        train_steps: кількість ітерацій градієнтного спуску
        learning_rate: швидкість навчання
    Вихід:
        R: матриця перетворення
    """
    np.random.seed(129)
    m = X.shape[0]
    n = X.shape[1]
    R = np.random.rand(n, n)

    for i in range(train_steps):
        if verbose and i % 10 == 0:
            print(f"Loss at iteration {i} is: {compute_loss(X, Y, R):.4f}")
        gradient = compute_gradient(X, Y, R)
        R -= learning_rate * gradient

    return R


In [19]:
R_train = align_embeddings(X_train, Y_train, train_steps=400, learning_rate=0.8)


Loss at iteration 0 is: 116.1960
Loss at iteration 10 is: 89.2659
Loss at iteration 20 is: 74.5666
Loss at iteration 30 is: 63.5847
Loss at iteration 40 is: 54.9662
Loss at iteration 50 is: 48.0138
Loss at iteration 60 is: 42.2934
Loss at iteration 70 is: 37.5143
Loss at iteration 80 is: 33.4727
Loss at iteration 90 is: 30.0209
Loss at iteration 100 is: 27.0481
Loss at iteration 110 is: 24.4699
Loss at iteration 120 is: 22.2202
Loss at iteration 130 is: 20.2466
Loss at iteration 140 is: 18.5070
Loss at iteration 150 is: 16.9671
Loss at iteration 160 is: 15.5987
Loss at iteration 170 is: 14.3783
Loss at iteration 180 is: 13.2865
Loss at iteration 190 is: 12.3066
Loss at iteration 200 is: 11.4248
Loss at iteration 210 is: 10.6292
Loss at iteration 220 is: 9.9095
Loss at iteration 230 is: 9.2570
Loss at iteration 240 is: 8.6640
Loss at iteration 250 is: 8.1241
Loss at iteration 260 is: 7.6315
Loss at iteration 270 is: 7.1813
Loss at iteration 280 is: 6.7689
Loss at iteration 290 is: 6.390

In [20]:
def realign_embeddings(X, Y, R, train_steps=100, learning_rate=0.0003, verbose=True,
                     compute_loss=compute_loss, compute_gradient=compute_gradient):
    """
    Вхід:
        X: матриця (розмірність m x n), кожен рядок - вектор слова мови-1
        Y: матриця (розмірність m x n), кожен рядок - вектор перекладу мовою-2
        R: матриця перетворення
        train_steps: кількість ітерацій градієнтного спуску
        learning_rate: швидкість навчання
    Вихід:
        R: матриця перетворення донавчена (fune tuned)
    """
    #np.random.seed(129)
    m = X.shape[0]
    n = X.shape[1]
    #R = np.random.rand(n, n)

    for i in range(train_steps):
        if verbose and i % 10 == 0:
            print(f"Loss at iteration {i} is: {compute_loss(X, Y, R):.4f}")
        gradient = compute_gradient(X, Y, R)
        R -= learning_rate * gradient

    return R


In [31]:
R_train = realign_embeddings(X_train, Y_train, R_train, train_steps=100, learning_rate=0.8)


Loss at iteration 0 is: 1.6023
Loss at iteration 10 is: 1.5741
Loss at iteration 20 is: 1.5471
Loss at iteration 30 is: 1.5213
Loss at iteration 40 is: 1.4966
Loss at iteration 50 is: 1.4730
Loss at iteration 60 is: 1.4503
Loss at iteration 70 is: 1.4285
Loss at iteration 80 is: 1.4077
Loss at iteration 90 is: 1.3876


In [33]:
R_train = realign_embeddings(X_train, Y_train, R_train, train_steps=500, learning_rate=0.8)


Loss at iteration 0 is: 1.3684
Loss at iteration 10 is: 1.3499
Loss at iteration 20 is: 1.3321
Loss at iteration 30 is: 1.3150
Loss at iteration 40 is: 1.2985
Loss at iteration 50 is: 1.2827
Loss at iteration 60 is: 1.2674
Loss at iteration 70 is: 1.2527
Loss at iteration 80 is: 1.2385
Loss at iteration 90 is: 1.2248
Loss at iteration 100 is: 1.2116
Loss at iteration 110 is: 1.1989
Loss at iteration 120 is: 1.1865
Loss at iteration 130 is: 1.1746
Loss at iteration 140 is: 1.1631
Loss at iteration 150 is: 1.1520
Loss at iteration 160 is: 1.1413
Loss at iteration 170 is: 1.1308
Loss at iteration 180 is: 1.1208
Loss at iteration 190 is: 1.1110
Loss at iteration 200 is: 1.1015
Loss at iteration 210 is: 1.0924
Loss at iteration 220 is: 1.0835
Loss at iteration 230 is: 1.0749
Loss at iteration 240 is: 1.0665
Loss at iteration 250 is: 1.0584
Loss at iteration 260 is: 1.0505
Loss at iteration 270 is: 1.0428
Loss at iteration 280 is: 1.0354
Loss at iteration 290 is: 1.0282
Loss at iteration 300

In [21]:
def cosine_similarity(A, B):
    cos = -10
    normb = np.linalg.norm(B, axis=1) # Calculate norm of each row in B

    if len(A.shape) == 1: # If A is a single vector
        norma = np.linalg.norm(A)
        dot = np.dot(A, B.T) # Calculate dot product with transpose of B
        epsilon = 1.0e-9 # to avoid division by 0
        cos = dot / (norma * normb + epsilon)
    else: # If A is a matrix
        norma = np.linalg.norm(A, axis=1)
        dot = np.dot(A, B.T) # Calculate dot product with transpose of B
        epsilon = 1.0e-9 # to avoid division by 0
        cos = dot / (norma * normb + epsilon)

    return cos

In [22]:
def translate(word, R, l1_embeddings, l2_embeddings, k=5):
    """
    Вхід:
        word: слово мови-1 для перекладу
        R: матриця перетворення
        l1_embeddings: KeyedVectors векторів слів мови-1
        l2_embeddings: KeyedVectors векторів слів мови-2
        k: кількість найближчих сусідів для пошуку
    Вихід:
        переклад слова мовою-2
    """
    if word not in l1_embeddings.key_to_index:
        return "Слово відсутнє у словнику"

    l1_emb = l1_embeddings[word]
    l2_emb = np.dot(l1_emb, R)
    distances = cosine_similarity(l2_emb, l2_embeddings.vectors)
    top_k = distances.argsort()[-k:][::-1]

    return l2_embeddings.index_to_key[top_k[0]]


In [39]:
R_train = realign_embeddings(X_train, Y_train, R_train, train_steps=500, learning_rate=2.0)


Loss at iteration 0 is: 0.7640
Loss at iteration 10 is: 0.7607
Loss at iteration 20 is: 0.7575
Loss at iteration 30 is: 0.7544
Loss at iteration 40 is: 0.7515
Loss at iteration 50 is: 0.7486
Loss at iteration 60 is: 0.7459
Loss at iteration 70 is: 0.7432
Loss at iteration 80 is: 0.7406
Loss at iteration 90 is: 0.7381
Loss at iteration 100 is: 0.7357
Loss at iteration 110 is: 0.7333
Loss at iteration 120 is: 0.7311
Loss at iteration 130 is: 0.7289
Loss at iteration 140 is: 0.7267
Loss at iteration 150 is: 0.7247
Loss at iteration 160 is: 0.7226
Loss at iteration 170 is: 0.7207
Loss at iteration 180 is: 0.7188
Loss at iteration 190 is: 0.7170
Loss at iteration 200 is: 0.7152
Loss at iteration 210 is: 0.7134
Loss at iteration 220 is: 0.7118
Loss at iteration 230 is: 0.7101
Loss at iteration 240 is: 0.7085
Loss at iteration 250 is: 0.7070
Loss at iteration 260 is: 0.7054
Loss at iteration 270 is: 0.7040
Loss at iteration 280 is: 0.7025
Loss at iteration 290 is: 0.7011
Loss at iteration 300

In [42]:
def align_embeddings_loss(X, Y, train_steps=100, learning_rate=0.0003, epsilon = 0.1, verbose=True,
                     compute_loss=compute_loss, compute_gradient=compute_gradient):
    """
    Вхід:
        X: матриця (розмірність m x n), кожен рядок - вектор слова мови-1
        Y: матриця (розмірність m x n), кожен рядок - вектор перекладу мовою-2
        train_steps: кількість ітерацій градієнтного спуску
        learning_rate: швидкість навчання
    Вихід:
        R: матриця перетворення
    """
    np.random.seed(129)
    m = X.shape[0]
    n = X.shape[1]
    R = np.random.rand(n, n)

    for i in range(train_steps):
        if verbose and i % 10 == 0:
            eps = compute_loss(X, Y, R)
            if eps < epsilon:
              break
            print(f"Loss at iteration {i} is: {eps:.4f}")
        gradient = compute_gradient(X, Y, R)
        R -= learning_rate * gradient

    return R


In [46]:
R_train = align_embeddings_loss(X_train, Y_train, train_steps=4000, learning_rate=5.0, epsilon = 0.6312)

Loss at iteration 0 is: 116.1960
Loss at iteration 10 is: 40.2906
Loss at iteration 20 is: 20.7850
Loss at iteration 30 is: 12.2932
Loss at iteration 40 is: 7.9696
Loss at iteration 50 is: 5.5476
Loss at iteration 60 is: 4.0938
Loss at iteration 70 is: 3.1726
Loss at iteration 80 is: 2.5625
Loss at iteration 90 is: 2.1429
Loss at iteration 100 is: 1.8448
Loss at iteration 110 is: 1.6268
Loss at iteration 120 is: 1.4634
Loss at iteration 130 is: 1.3379
Loss at iteration 140 is: 1.2397
Loss at iteration 150 is: 1.1613
Loss at iteration 160 is: 1.0977
Loss at iteration 170 is: 1.0454
Loss at iteration 180 is: 1.0018
Loss at iteration 190 is: 0.9650
Loss at iteration 200 is: 0.9337
Loss at iteration 210 is: 0.9067
Loss at iteration 220 is: 0.8832
Loss at iteration 230 is: 0.8627
Loss at iteration 240 is: 0.8447
Loss at iteration 250 is: 0.8287
Loss at iteration 260 is: 0.8144
Loss at iteration 270 is: 0.8017
Loss at iteration 280 is: 0.7902
Loss at iteration 290 is: 0.7798
Loss at iteratio

In [47]:
translate("ukrainian", R_train, lang1_embeddings, lang2_embeddings)

'українська'

NER - Name Entity Recognition

In [48]:
phrase = """
first page article made discussion
"""
phrase_list = [line.strip() for line in phrase.strip().split(' ')]

result = ""
for word in phrase_list:
  result = result + " " + translate(word, R_train, lang1_embeddings, lang2_embeddings)
print("Переклад: ", result)

Переклад:   перший сторінку інтернет-блог зроблено обговорювання


In [ ]:
phrase_list

['Melissa is now passing through the Bahamas as a Category 1 hurricane after making landfall in Cuba this morning']

In [49]:
def test_vocabulary(X, Y, R, l1_embeddings, l2_embeddings):
    """
    Вхід:
        X: матриця векторів слів мови-1
        Y: матриця векторів слів мови-2
        R: матриця перетворення
        l1_embeddings: KeyedVectors векторів слів мови-1
        l2_embeddings: KeyedVectors векторів слів мови-2
    Вихід:
        accuracy: точність перекладу
    """
    l1_words = list(l1_embeddings.index_to_key)
    l2_words = list(l2_embeddings.index_to_key)

    pred = np.dot(X, R)
    correct = 0
    for i in range(len(X)):
        l1_word = l1_words[i]
        pred_l2 = translate(l1_word, R, l1_embeddings, l2_embeddings, k=1)
        true_l2 = l2_words[i]
        if pred_l2 == true_l2:
            correct += 1

    accuracy = correct / len(X)
    return accuracy



In [50]:
X_test, Y_test = get_matrices(l1_l2_test, lang2_embeddings, lang1_embeddings)


In [51]:
X_test.shape

(1286, 300)

In [ ]:
accuracy_test = test_vocabulary(X_test, Y_test, R_train, lang1_embeddings, lang2_embeddings)
print(f"Точність перекладу на тестовому наборі: {accuracy_test:.3f}")


In [54]:
import nltk
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [55]:
from nltk.corpus import twitter_samples

all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
all_tweets = all_positive_tweets + all_negative_tweets


In [65]:
len(all_tweets)

10000

In [56]:
all_tweets[0]

'#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)'

In [61]:
import re, string

In [62]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer


In [60]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [57]:
def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


In [73]:
def get_tweet_embedding(tweet, l1_embeddings):
    """
    Вхід:
        tweet: рядок з текстом твіта
        l1_embeddings: KeyedVectors векторів слів
    Вихід:
        tweet_embedding: вектор ознак для твіта
    """
    words = process_tweet(tweet)
    embeddings = [l1_embeddings[w] if w in l1_embeddings.key_to_index else np.zeros(300) for w in words]
    if not embeddings:  # Handle empty embeddings list
        return np.zeros(300)
    tweet_embedding = np.mean(embeddings, axis=0)
    return tweet_embedding

def get_all_tweet_embeddings(tweets, l1_embeddings):
    """
    Вхід:
        tweets: список твітів
        l1_embeddings: словник векторів слів
    Вихід:
        tweet_embeddings: словник {id_твіта: вектор_твіта}
        tweet_id_map: список id твітів
    """
    tweet_embeddings = {}
    tweet_id_map = []
    for i, tweet in enumerate(tweets):
        tweet_embeddings[i] = get_tweet_embedding(tweet, l1_embeddings)
        tweet_id_map.append(i)
    return tweet_embeddings, tweet_id_map

In [74]:
tweet_embeddings, tweet_id_map = get_all_tweet_embeddings(all_tweets, lang1_embeddings)
print(f"Кількість твітів: {len(tweet_id_map)}")
print(f"Розмірність векторів твітів: {tweet_embeddings[0].shape}")


Кількість твітів: 10000
Розмірність векторів твітів: (300,)


In [75]:
def hash_func(embedding, planes):
    """
    Вхід:
        embedding: вектор ознак (розмірність 1 x n)
        planes: матриця випадкових гіперплощин (розмірність n x n_planes)
    Вихід:
        hash_value: хеш-значення
    """
    # Ensure embedding is a 2D array for matrix multiplication
    embedding = embedding.reshape(1, -1)
    dots = np.dot(embedding, planes)
    # Get the sign of the dot product
    signs = np.sign(dots)
    # Convert signs to 0 or 1
    binary_signs = (signs > 0).astype(int)
    # Calculate the hash value
    hash_value = 0
    for i in range(binary_signs.shape[1]):
        hash_value += binary_signs[0, i] * (2 ** i)
    return hash_value

In [76]:
def make_hash_table(embeddings, planes):
    """
    Вхід:
        embeddings: набір векторів ознак
        planes: матриця випадкових гіперплощин
    Вихід:
        hash_table: хеш-таблиця у вигляді словника:
                    {хеш_значення: [список векторів з цим хеш_значенням]}
    """
    hash_table = {}
    for i, embedding in enumerate(embeddings):
        hash_value = hash_func(embedding, planes)
        hash_table.setdefault(hash_value, []).append(i)
    return hash_table


In [77]:
def init_lsh(embeddings, n_planes, n_tables):
    """
    Вхід:
        embeddings: набір векторів ознак
        n_planes: кількість гіперплощин
        n_tables: кількість хеш-таблиць
    Вихід:
        tables: список хеш-таблиць
        planes_list: список матриць випадкових гіперплощин
    """
    planes_list = [np.random.normal(size=(embeddings.shape[1], n_planes))
                   for _ in range(n_tables)]
    tables = [make_hash_table(embeddings, planes) for planes in planes_list]
    return tables, planes_list


In [78]:
tables, planes_list = init_lsh(np.array(list(tweet_embeddings.values())), n_planes=10, n_tables=10)


In [79]:
tables


[{np.int64(728): [0,
   10,
   51,
   69,
   105,
   113,
   154,
   163,
   164,
   195,
   209,
   243,
   253,
   272,
   321,
   501,
   535,
   557,
   602,
   708,
   738,
   782,
   900,
   957,
   1016,
   1030,
   1194,
   1401,
   1952,
   1987,
   1988,
   2202,
   2647,
   2831,
   3042,
   3171,
   3411,
   3754,
   3847,
   3907,
   3999,
   4135,
   4234,
   4452,
   4559,
   4584,
   4750,
   5833,
   6402,
   6540,
   8204],
  np.int64(709): [1,
   96,
   141,
   166,
   703,
   901,
   1592,
   1655,
   1858,
   1867,
   2175,
   2410,
   2838,
   3026,
   3234,
   3308,
   3337,
   3609,
   3701,
   3747,
   3806,
   3986,
   4087,
   4608,
   4656,
   4758,
   4784,
   4791,
   4793,
   4887,
   4919,
   4948,
   4949,
   4952,
   4956,
   4995,
   5068,
   5125,
   5215,
   5272,
   5410,
   5413,
   5465,
   5486,
   5558,
   5570,
   5627,
   5644,
   5682,
   5696,
   5952,
   5980,
   6001,
   6008,
   6033,
   6066,
   6072,
   6288,
   6367,
   6393,
   6511,

In [82]:
len(planes_list)

10

In [84]:
planes_list[0].shape

(300, 10)

In [85]:
def lsh_knn(tweet_id, embedding, tables, planes_list, k=5):
    """
    Вхід:
        tweet_id: індекс твіта
        embedding: вектор ознак твіта
        tables: список хеш-таблиць
        planes_list: список матриць випадкових гіперплощин
        k: кількість найближчих сусідів
    Вихід:
        neighbors: список індексів k найближчих сусідів
    """
    candidates = set()
    for table, planes in zip(tables, planes_list):
        hash_value = hash_func(embedding, planes)
        candidates.update(table.get(hash_value, []))

    candidates.discard(tweet_id)
    candidates = list(candidates)
    candidate_embeddings = np.array([tweet_embeddings[i] for i in candidates])

    distances = cosine_similarity(embedding.reshape(1, -1), candidate_embeddings)
    top_k_idx = np.argsort(distances.squeeze())[-k:][::-1]
    neighbors = [candidates[i] for i in top_k_idx]

    return neighbors


In [99]:
tweet_id = 0 # чи якийсь інший номер або просто уведений користувачем
#tweet = all_tweets[tweet_id]
tweet = """
The Guardian has only ever published 15 zero-star reviews. Here they all are
As Kim Kardashian’s All’s Fair sets a new low for TV, we revisit every single thing our critics have mercilessly panned. Brace yourself for the Mount Rushmore of rubbish!

Lucy Mangan’s Guardian review of Kim Kardashian’s new Disney+ legal drama All’s Fair was something of a rarity. Not necessarily because she didn’t care for it – the scorn has been universal – but because she gave it zero stars.
"""

In [100]:
print(f"Запит: {tweet}")

embedding = get_tweet_embedding(tweet, lang1_embeddings)


Запит: 
The Guardian has only ever published 15 zero-star reviews. Here they all are
As Kim Kardashian’s All’s Fair sets a new low for TV, we revisit every single thing our critics have mercilessly panned. Brace yourself for the Mount Rushmore of rubbish!

Lucy Mangan’s Guardian review of Kim Kardashian’s new Disney+ legal drama All’s Fair was something of a rarity. Not necessarily because she didn’t care for it – the scorn has been universal – but because she gave it zero stars.



In [101]:
embedding

array([-1.94942311e-02, -2.90865380e-02, -4.02000000e-02,  3.39403852e-02,
        6.64211537e-02,  2.27499994e-03, -1.21230769e-02, -4.16769226e-02,
        1.99153849e-02,  4.72096150e-02, -8.89038431e-03,  2.50480778e-02,
        3.22000009e-02,  8.26925305e-05,  1.27230763e-02,  7.05788428e-02,
        2.82576929e-02,  1.72115386e-02, -4.08365380e-02,  2.54980767e-02,
        7.25576927e-03,  4.83846160e-02,  7.84423086e-03, -2.67038444e-02,
        2.03749998e-02, -6.12307686e-02, -1.36980769e-02, -2.04288457e-02,
       -3.22000006e-02,  1.78680768e-01, -9.75192281e-03, -4.23750005e-02,
       -2.78250004e-02,  6.16538457e-03,  1.29346157e-02, -7.37500007e-03,
       -5.72500003e-03, -1.31653845e-02, -4.03249994e-02,  5.75192298e-03,
        1.35384618e-02, -8.94807696e-03, -3.76903859e-02,  6.41769211e-02,
        4.44230812e-04,  4.34692318e-02, -9.84038473e-03,  2.66288464e-02,
       -4.07884625e-03,  3.34423129e-03,  2.36019235e-02,  1.66923070e-03,
       -9.79038471e-03, -

In [102]:
neighbors = lsh_knn(tweet_id, embedding, tables, planes_list, k=3)

print("Найбільш схожі твіти:")
for neighbor_id in neighbors:
    print(all_tweets[neighbor_id])


Найбільш схожі твіти:
I’d really like to handle Murielle Ahouré’s PR/Brand Image. She’s  missing too many opportunities
 :(
. @Ben_Barker1989 yeah doesn’t appear to be working. But I want a free cornetto :( (not a strawberry one, they’re a fake cornetto)
Rape time at it’s best :D http://t.co/fC5U1TusUk &gt;&gt; http://t.co/GBxQmizM7O http://t.co/0Um6xgpsAn
